# Weights Initialization

## 1 背景介绍   

神经网络的训练过程，就是自动调整网络中参数的过程。在训练的起初，网络的参数总要从某一状态开始，而这个初始状态的设定，就是神经网络的初始化，而且正确的参数初始化对模型的收敛至关重要。  

合适的网络初始值，不仅有助于梯度下降法在一个好的“起点”上去寻找最优值，还能避免神经元发生学习饱和。  

这里说的是权重W的初始化，偏置（biases）一般直接初始化为 0 就可以。

## 2 全零初始化（错误）  

用梯度下降法训练神经网络时, 主要从两点考虑参数初始化的方法：一: 最终是否能得到想要的学习结果, 即是否能得到一个符合预期目标的模型；二: 训练时间, 好的参数初始化可以有效缩短训练时间。  

将所有参数都初始化为0是最省力的做法。有些情况下可行, 但大部分情况下会导致学习失败, 得不到可用的模型。  

可行的情况：  

没有隐藏层的神经网络，即训练单层神经网络时候，可以简单的将权重初始化为全 0 数据，$ f(x) = w^Tx + b $可以看出当$ w = [0,0,...,0] $的时候，$ dw = x$, 只要输入 x 不为0，则经过一次反向传播，w 和 b 会更新为非 0 值，从而可以使训练过程继续下去。  

不可行的情况：  

在训练多层神经网络时候，这种初始化方式是错误的，因为如果网络中的每个神经元都计算出同样的输出，然后它们就会在反向传播中计算出同样的梯度，从而进行同样的参数更新。换句话说，如果权重被初始化为同样的值，神经元之间就失去了不对称性的源头，因为如果所有的参数都是0，那么所有神经元的输出都将是相同的，那在 back propagation 的时候同一层内所有神经元的行为也是相同的 gradient 相同，weight update 也相同，可以通过以下实验，观察参数初始化为0时，权重更新的情况。

In [ ]:
""" Multilayer Perceptron.
A Multilayer Perceptron (Neural Network) implementation example using
TensorFlow library. This example is using the MNIST database of handwritten
digits (http://yann.lecun.com/exdb/mnist/).
Links:
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

# ------------------------------------------------------------------
#
# THIS EXAMPLE HAS BEEN RENAMED 'neural_network.py', FOR SIMPLICITY.
#
# ------------------------------------------------------------------


from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/home/liyibo/demo/MNIST_data", one_hot=True)
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape, mnist.validation.labels.shape)

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 1
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.zeros([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.zeros([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_11 = tf.nn.sigmoid(layer_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_11, weights['h2']), biases['b2'])
    layer_22 = tf.nn.sigmoid(layer_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_22, weights['out']) + biases['out']
    # out_layer = tf.matmul(layer_11, weights['out']) + biases['out']
    out_layer2 = tf.nn.sigmoid(out_layer)
    return out_layer2

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

variables = tf.contrib.slim.get_trainable_variables()
grads_and_vars = optimizer.compute_gradients(loss_op, variables)
train_op = optimizer.apply_gradients(grads_and_vars)

# train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            weights1,grads_and_vars2,_, c = sess.run([weights, grads_and_vars, train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

## 3 可行的初始化方法  

### 3.1 pre-training  

pre-training 是早期训练神经网络的有效初始化方法，一般先对每层参数进行无监督预训练，然后再在当前任务上做 fine-tuning。具体过程如下所示：（1）pre-training 阶段，将网络模型中的每一层取出，构造一个 auto-encoder 做训练，使得输入层和输出层保持一致。在这一过程中，参数得以更新，形成初始值；（2）fine-tuning 阶段，将 pre-train 过的每一层放回神经网络，利用 pre-train 阶段得到的参数初始值和训练数据对模型进行整体调整，在这一过程中，参数进一步被更新，形成最终模型。

随着数据量的增加及各种激活函数的提出，pre-training 的概念已经发生变化。例如在检测任务上，一般会引入在分类任务上训练好的模型，将该模型的参数作为检测任务模型的初始化参数进行训练，可以加快收敛，提升效果。  

### 3.2 random initialization  

随机初始化是很多人目前经常使用的方法，然而这是有弊端的，一旦随机分布选择不当，就会导致网络优化陷入困境。下面举几个例子。  

1、W = 0.01 * np.random.randn(node_in, node_out)   

参数初始化为均值为 0，标准差为 0.01 的随机数。随着层数的增加，这种初始化会导致有些层的输出集中在 0 附近，在计算这些层的梯度的时候，输出值是计算梯度的乘法因子，就会导致计算出的梯度很小，没办法更新参数，进行训练。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/w_init_1.jpg?raw=true)
<center> ** 0.01 方差初始化** </center >

2、W = np.random.randn(node_in, node_out)  

参数初始化为均值为0，标准差为 1 的随机数。这种情况下输出值集中在 -1 或 1 附近，神经元容易饱和，同样会导致了梯度很小，参数难以被更新。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/w_init_2.jpg?raw=true)
<center> ** 1 方差初始化** </center >

### 3.3 Xavier initialization  

上面做法存在一个问题，随着输入数据量的增长，随机初始化的神经元的输出数据的分布中的方差也在增大。可以除以输入数据量的平方根来调整其数值范围，这样神经元输出的方差就归一化到 1 了。通过Xavier 初始化就保证了网络中所有神经元起始时有近似同样的输出分布。实践经验证明，这样做可以提高收敛的速度。  

Xavier 初始化的基本思想是保持输入和输出的方差一致，避免了所有输出值都趋向于0，输出值在很多层之后能够保持着良好的分布，可以解决上面的问题。  

W = 0.01 * np.random.randn(node_in, node_out) / np.sqrt(node_in)  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/w_init_3.jpg?raw=true)
<center> ** Xavier 初始化(tanh)** </center >

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/w_init_4.jpg?raw=true)
<center> ** Xavier 初始化(relu)** </center >

### 3.4 He initialization  

He initialization 的思想是：在ReLU网络中，假定每一层有一半的神经元被激活，另一半为 0，所以，要保持 variance 不变，只需要在 Xavier 的基础上再除以2。  

W = 0.01 * np.random.randn(node_in, node_out) / np.sqrt(node_in/2)  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/w_init_5.jpg?raw=true)
<center> ** He 初始化** </center >

## 4 参考  

1 [聊一聊深度学习的weight initialization](https://zhuanlan.zhihu.com/p/25110150)  
2 [贺完结！CS231n官方笔记授权翻译总集篇发布](https://zhuanlan.zhihu.com/p/21930884?refer=intelligentunit)  